# Book 3 - Otimização Modelos de Machine Learning

Este notebook serviu como registro prático e teórico no meu aprendizado de Machine Learning.

`Enriqueci este notebook com anotações adicionais e aplicações práticas tornando-o uma referência valiosa para consultas e implementações em futuros projetos reais.`

Espero que este material inspire outros a explorar ainda mais o fascinante mundo do Machine Learning. 

No notebook presente tem todos os topicos dos notebook anteriores, porém sendo acrescentado e aprofundado com anotações dos seguintes tópicos:

- **Busca em Grade (Grid Search)**: Otimização de Hiperterparâmetros
- **Validação Cruzada Aninhada (Nested Cross Validation)**
- **Randomized Search CV (Busca Aleatória)**: Otimização de Hiperparâmetros
- **Otimização Bayesiana**: Otimização de Hiperparâmetros

Compartilhar conhecimento é uma alegria—viva ao aprendizado contínuo, boa pratica e bons estudo a quem estiver lendo, abraços!

# Funções, bibliotecas e Dataframe ficticios

In [1]:
RANDOM_STATE = 3141592

In [2]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib inline

# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
import requests

# CARREGANDO BASE
arquivo = 'fake_database'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
response = requests.get(url)
code = response.text
exec(code)
df = fake_database2()
display(df.head())

arquivo = 'funcoes_estatisticas'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
response = requests.get(url)
code = response.text
exec(code)
print('TUDO OK')

,x_num0,x_num1,x_num2,x_num3,x_num4,x_num5,x_num6,x_num7,x_num8,x_num9,...,x_bin0,x_bin1,x_bin2,x_bin3,x_bin4,x_cat0,x_cat1,x_cat2,x_cat_0,x_cat_1
0,4.875997,-1.491729,2.491069,1.874948,-0.000381,1.478758,3.972563,0.230329,5.149503,0.595726,...,sim,não,sim,não,sim,A,B,A,C,D
1,3.750108,-0.510656,-0.462908,0.928715,0.496968,-0.995687,2.219878,-1.951839,0.335021,-0.071709,...,não,não,não,sim,não,B,B,C,C,B
2,1.915908,2.793605,2.989653,1.743696,-0.213394,-0.274895,0.106642,1.605156,2.169961,-1.126328,...,sim,sim,sim,não,sim,C,A,C,B,C
3,3.575351,2.779022,4.928382,3.046386,-1.832086,-2.961492,1.171947,1.044997,3.281876,-1.733729,...,não,não,não,sim,não,B,A,A,D,D
4,4.812294,0.316227,0.704474,0.908321,0.050424,-2.080064,2.597434,-1.643092,-0.138313,-1.498914,...,sim,não,não,não,não,A,B,A,D,B


TUDO OK


In [4]:
# Variavel Dependente
var_dep = 'y'
y = df[var_dep]
display(y.head())

x = df.drop('y', axis=1)
display(x.head())

0    0
1    1
2    0
3    1
4    1
Name: y, dtype: object

,x_num0,x_num1,x_num2,x_num3,x_num4,x_num5,x_num6,x_num7,x_num8,x_num9,x_bin0,x_bin1,x_bin2,x_bin3,x_bin4,x_cat0,x_cat1,x_cat2,x_cat_0,x_cat_1
0,4.875997,-1.491729,2.491069,1.874948,-0.000381,1.478758,3.972563,0.230329,5.149503,0.595726,sim,não,sim,não,sim,A,B,A,C,D
1,3.750108,-0.510656,-0.462908,0.928715,0.496968,-0.995687,2.219878,-1.951839,0.335021,-0.071709,não,não,não,sim,não,B,B,C,C,B
2,1.915908,2.793605,2.989653,1.743696,-0.213394,-0.274895,0.106642,1.605156,2.169961,-1.126328,sim,sim,sim,não,sim,C,A,C,B,C
3,3.575351,2.779022,4.928382,3.046386,-1.832086,-2.961492,1.171947,1.044997,3.281876,-1.733729,não,não,não,sim,não,B,A,A,D,D
4,4.812294,0.316227,0.704474,0.908321,0.050424,-2.080064,2.597434,-1.643092,-0.138313,-1.498914,sim,não,não,não,não,A,B,A,D,B


In [5]:
# DUMMYRIZAÇÃO
colunas_categoricas = []
colunas_binarias = []
colunas_mais3_categorias = []

for coluna in x.columns:
    if df[coluna].dtype == 'O':
        categorias = x[coluna].unique()
        if len(categorias) == 2:
            print('2 niveis:', coluna, '=>', categorias)
            colunas_categoricas.append(coluna)
            colunas_binarias.append(coluna)
        else:
            print('3 niveis:', coluna, '=>', categorias)
            colunas_categoricas.append(coluna)
            colunas_mais3_categorias.append(coluna)

############################################################################################
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder #transformando colunas com 2 categorias em 0 e 1

coluna = x.columns
one_hot = make_column_transformer((
    OneHotEncoder(drop='if_binary'), #caso a coluna tenha apenas 2 categorias 
    colunas_categoricas), #passando quais são essas colunas
    remainder = 'passthrough', sparse_threshold=0) #oque deve ser feito com as outras

#Aplicando transformação
x = one_hot.fit_transform(x)

#Os novos nomes das colunas #'onehotencoder=transformadas; 'remainder'=não transformadas
novos_nomes_colunas = one_hot.get_feature_names_out(coluna)

x = pd.DataFrame(x, columns = novos_nomes_colunas) #alterando de volta
x_columns = x.columns.tolist() 

############################################################################################
# NORMALIZAÇÃO
from sklearn.preprocessing import MinMaxScaler
normalizacao = MinMaxScaler()
x = normalizacao.fit_transform(x)

#df['Close_padronizada'] = (df[coluna] - df[coluna].mean()) / df[coluna].std()
#df['Close_normalizada'] = (df[coluna] - df[coluna].min()) / (df[coluna].max() - df[coluna].min())

############################################################################################
# DEFININDO A VARIAVEL DEPENDENTE
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

############################################################################################
x_inteiro = x
y_inteiro = y

# DIVIDINDO BASE EM TREINO E TESTE
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, 
                                                    stratify = y, #para manter a proporção da Var Dep nos splits
                                                    random_state = 5) #raiz da aleatoridade
# test_size = 0.25 #porcentagem que ira ser separado para testes

print(x_treino.shape, x_teste.shape)
print(y_treino.shape, y_teste.shape)

2 niveis: x_bin0 => ['sim' 'não']
2 niveis: x_bin1 => ['não' 'sim']
2 niveis: x_bin2 => ['sim' 'não']
2 niveis: x_bin3 => ['não' 'sim']
2 niveis: x_bin4 => ['sim' 'não']
3 niveis: x_cat0 => ['A' 'B' 'C']
3 niveis: x_cat1 => ['B' 'A' 'C']
3 niveis: x_cat2 => ['A' 'C' 'B']
3 niveis: x_cat_0 => ['C' 'B' 'D' 'A']
3 niveis: x_cat_1 => ['D' 'B' 'C' 'A']
(750, 32) (250, 32)
(750,) (250,)


In [6]:
# Função para avaliação de modelos exibindo metricas de avaliação
import requests
arquivo = 'ML_supervised_learning'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
response = requests.get(url)
code = response.text
exec(code)

# =======================================

# Modelos usados

In [7]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

# Regressão Logistica
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#classificador = make_pipeline(StandardScaler(), LogisticRegression(random_state=RANDOM_STATE))

# Busca em Grade (Grid Search) - Otimização de Hiperparâmetros

A Busca em Grade, ou Grid Search, é uma técnica comum para otimização de hiperparâmetros em modelos de aprendizado de máquina. Este método sistemático testa exaustivamente uma série de combinações de hiperparâmetros para determinar a configuração ideal que resulta no melhor desempenho do modelo.

- **Exaustividade:** Ao testar todas as possíveis combinações, garante-se que a melhor configuração dentro do espaço definido será encontrada.

**Funcionamento**

1. **Definição do Espaço de Parâmetros:**
   Antes de iniciar a busca, é necessário definir o "grid" de hiperparâmetros, que é essencialmente uma lista de valores a serem testados para cada hiperparâmetro do modelo.

2. **Avaliação Exaustiva:**
   Cada combinação possível de hiperparâmetros no grid é testada. O modelo é treinado com cada conjunto de hiperparâmetros, e seu desempenho é avaliado usando uma métrica específica, geralmente através de validação cruzada.

3. **Seleção do Melhor Modelo:**
   Depois de testar todas as combinações, a configuração que proporciona o melhor desempenho de acordo com a métrica escolhida é selecionada como a ideal.

**Alternativas**

- **Busca Aleatória (Random Search):** Ao invés de testar todas as combinações possíveis, a Busca Aleatória seleciona um número fixo de combinações aleatoriamente, reduzindo o tempo necessário para a busca.
- **Métodos Baseados em Gradiente:** Técnicas como otimização bayesiana usam modelos probabilísticos para selecionar inteligentemente os hiperparâmetros que têm maior probabilidade de melhorar o desempenho do modelo.

## in Decision Tree

In [8]:
from sklearn.model_selection import GridSearchCV

# Definindo grade de hiperparametros
param_grid_dt = {
    'criterion': ['gini', 'entropy'], # a função que vai decidir a qualidade de uma divisão
    'max_depth': np.linspace(6, 12, 4, dtype=int), # profundidade da arvore
    'min_samples_split': np.linspace(5, 20, 4, dtype=int), # quantidade minima de amostra para dividir um nó interno
    'min_samples_leaf': np.linspace(5, 20, 4, dtype=int), # quantidade minima de amostra para estar em uma folha
    'max_features': ['sqrt', 'log2'], # quantidade máxima de features que devem estar na divisão de um nó
    'splitter': ['best', 'random'] # mede se a forma de divisão é a melhor ou aleatória.
}

# Cross Validation -> CV estratificado
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, 
                     shuffle=True, # embarelhamento dos dados
                     random_state=RANDOM_STATE)

grid_search = GridSearchCV(
    estimator = DecisionTreeClassifier(random_state=RANDOM_STATE), #classificador - modelo
    param_grid = param_grid_dt,  
    scoring='recall', # metrica que sera avaliada
    n_jobs=-1, # quantidade de processadores para usar da maquina #-1 usa todos os disponiveis
    refit=True, # retreinar com os melhores hiper parametros
    cv=cv, # validação cruzada #
    verbose=0, # mede a quantidade de informações que sera retornada # 0 é o padrão
    pre_dispatch='2*n_jobs', #
    #error_score=nan, #
    return_train_score=False) # scores de treinamento

# Treinando modelo
grid_search.fit(x_treino, y_treino)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=3141592, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=3141592), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 6,  8, 10, 12]),
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': array([ 5, 10, 15, 20]),
                         'min_samples_split': array([ 5, 10, 15, 20]),
                         'splitter': ['best', 'random']},
             scoring='recall')

In [9]:
cv_results = pd.DataFrame(grid_search.cv_results_)
display(cv_results.head().T)
###################################################
# melhores hiper-parametros
#display(grid_search.best_params_)

#  melhor modelo
display(cv_results.loc[[grid_search.best_index_]].T)

,0,1,2,3,4
mean_fit_time,0.005788,0.00162,0.002497,0.001529,0.002139
std_fit_time,0.001578,0.000467,0.000632,0.000481,0.000199
mean_score_time,0.0035,0.002577,0.001743,0.002171,0.00204
std_score_time,0.001789,0.000597,0.000387,0.000423,0.000268
param_criterion,gini,gini,gini,gini,gini
param_max_depth,6,6,6,6,6
param_max_features,sqrt,sqrt,sqrt,sqrt,sqrt
param_min_samples_leaf,5,5,5,5,5
param_min_samples_split,5,5,10,10,15
param_splitter,best,random,best,random,best


,71
mean_fit_time,0.000756
std_fit_time,0.000637
mean_score_time,0.00211
std_score_time,0.001367
param_criterion,gini
param_max_depth,8
param_max_features,sqrt
param_min_samples_leaf,5
param_min_samples_split,20
param_splitter,random


In [10]:
num_cat_hip(cv_results, 'param_max_depth', 'mean_test_score')

P-value (Kruskal-Wallis): 0.97


In [11]:
num_cat_hip(cv_results, 'param_min_samples_leaf', 'mean_test_score')

P-value (Kruskal-Wallis): < 0.001
P-value (Pós-Teste de Dunn):
          5       10       15       20
5      1.00  < 0.001  < 0.001  < 0.001
10  < 0.001     1.00     1.00  < 0.001
15  < 0.001     1.00     1.00  < 0.001
20  < 0.001  < 0.001  < 0.001     1.00


None

## in Logistic Regression

In [12]:
max_iter = np.linspace(100, 300, 5, dtype=int)
c = [0.001, 0.01, 0.1, 1, 10]

param_grid_lr = [  # Define duas grades distintas de hiperparâmetros
    {   # Primeira grade para os solvers newton-cg e lbfgs
        'logisticregression__solver': ['newton-cg', 'lbfgs'],  # Solvers para otimização
        'logisticregression__penalty': ['l2'],  # Penalidade L2 para regularização
        'logisticregression__max_iter': max_iter,  # Número máximo de iterações para a convergência do solver
        'logisticregression__C': c  # Parâmetro de regularização inverso; valores menores especificam regularização mais forte
    },
    {   # Segunda grade para o solver liblinear
        'logisticregression__solver': ['liblinear'],  # Solver que suporta regularização L1
        'logisticregression__penalty': ['l1', 'l2'],  # Penalidades L1 e L2 para regularização
        'logisticregression__max_iter': max_iter,  # Número máximo de iterações para a convergência do solver
        'logisticregression__C': c  # Parâmetro de regularização inverso; valores menores especificam regularização mais forte
    },
]

from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, 
                     shuffle=True, # embarelhamento dos dados
                     random_state=RANDOM_STATE)

grid_search = GridSearchCV(
    estimator = make_pipeline(StandardScaler(), LogisticRegression()),
    param_grid = param_grid_lr,
    scoring = "recall",
    n_jobs = -1,
    cv = cv)

grid_search.fit(x_treino, y_treino)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=3141592, shuffle=True),
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('logisticregression',
                                        LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'logisticregression__C': [0.001, 0.01, 0.1, 1, 10],
                          'logisticregression__max_iter': array([100, 150, 200, 250, 300]),
                          'logisticregression__penalty': ['l2'],
                          'logisticregression__solver': ['newton-cg', 'lbfgs']},
                         {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10],
                          'logisticregression__max_iter': array([100, 150, 200, 250, 300]),
                          'logisticregression__penalty': ['l1', 'l2'],
                          'logisticregression__solver': ['liblinear']}],
             scoring='recall')

In [13]:
cv_results = pd.DataFrame(grid_search.cv_results_)
display(cv_results.head().T)
###################################################
# melhores hiper-parametros
#display(grid_search.best_params_)

#  melhor modelo
display(cv_results.loc[[grid_search.best_index_]].T)

,0,1,2,3,4
mean_fit_time,0.018525,0.013237,0.003163,0.00208,0.002663
std_fit_time,0.001602,0.006835,0.000832,0.003277,0.003378
mean_score_time,0.0016,0.001503,0.00084,0.0,0.003565
std_score_time,0.00049,0.000449,0.001141,0.0,0.003402
param_logisticregression__C,0.001,0.001,0.001,0.001,0.001
param_logisticregression__max_iter,100,100,150,150,200
param_logisticregression__penalty,l2,l2,l2,l2,l2
param_logisticregression__solver,newton-cg,lbfgs,newton-cg,lbfgs,newton-cg
params,"{'logisticregression__C': 0.001, 'logisticregr...","{'logisticregression__C': 0.001, 'logisticregr...","{'logisticregression__C': 0.001, 'logisticregr...","{'logisticregression__C': 0.001, 'logisticregr...","{'logisticregression__C': 0.001, 'logisticregr..."
split0_test_score,0.921053,0.921053,0.921053,0.921053,0.921053


,0
mean_fit_time,0.018525
std_fit_time,0.001602
mean_score_time,0.0016
std_score_time,0.00049
param_logisticregression__C,0.001
param_logisticregression__max_iter,100
param_logisticregression__penalty,l2
param_logisticregression__solver,newton-cg
params,"{'logisticregression__C': 0.001, 'logisticregr..."
split0_test_score,0.921053


In [14]:
num_cat_hip(cv_results, 'param_logisticregression__max_iter', 'mean_test_score')

P-value (Kruskal-Wallis): 1.00


# Nested Cross Validation - Validação Cruzada Aninhada

A validação cruzada aninhada é uma extensão da validação cruzada tradicional que ajuda a obter uma estimativa mais realista e menos otimista do erro de generalização de um modelo. Esta técnica é particularmente útil quando o objetivo é selecionar tanto o modelo quanto os hiperparâmetros de forma mais precisa.

Isso porque ela permite que cada combinação de hiperparâmetros seja avaliada de forma mais imparcial, usando uma parte do conjunto de dados que não foi "vista" pelos hiperparâmetros durante sua seleção.

- **Estimativa Imparcial do Desempenho:** Ao separar a seleção de hiperparâmetros do teste final, a validação cruzada aninhada fornece uma estimativa mais imparcial e geralmente mais conservadora do desempenho do modelo.
- **Validade Estatística:** Ajustar os hiperparâmetros de forma independente para cada fold externo ajuda a evitar o viés e o sobreajuste, aumentando a validade estatística dos resultados.

## Validação Cruzada Aninhada vs. Não Aninhada

**Validação Cruzada Não Aninhada**

Na validação cruzada não aninhada (ou padrão), dividimos o conjunto de dados em \( k \) folds e realizamos \( k \) iterações de treinamento e teste. Em cada iteração, um fold diferente é usado como teste, e os restantes são usados para treinamento. Este método é eficaz para avaliar o desempenho do modelo, mas tem algumas limitações:
- **Otimismo no Desempenho:** Como o mesmo conjunto de dados é usado para selecionar os hiperparâmetros e avaliar o modelo, pode haver um viés otimista no desempenho estimado.
- **Dependência de Hiperparâmetros:** A escolha dos hiperparâmetros pode influenciar significativamente a estimativa de desempenho, resultando em uma possível sobreajuste à configuração particular do conjunto de dados de teste.

**Validação Cruzada Aninhada**

A validação cruzada aninhada resolve essas questões utilizando dois níveis de validação cruzada:
- **Validação Cruzada Externa:** Divide o conjunto de dados em \( k \) folds externos. Cada fold externo serve uma vez como conjunto de teste, enquanto os folds restantes são usados para a validação cruzada interna.
- **Validação Cruzada Interna:** Dentro de cada fold externo, realiza-se uma segunda validação cruzada para selecionar os hiperparâmetros. Isso significa que os hiperparâmetros são ajustados independentemente para cada fold externo.

## in Decision Tree

In [15]:
# Validação internar
inner_cv = StratifiedKFold(shuffle=True, random_state=RANDOM_STATE)

# Validação externar
outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

In [16]:
from sklearn.model_selection import cross_val_score

grid_search = GridSearchCV(estimator = DecisionTreeClassifier(random_state=RANDOM_STATE),
    param_grid = param_grid_dt,
    scoring = "recall",
    n_jobs = -1,
    cv = inner_cv
    )

# Validação cruzada externa
dt_nested_scores = cross_val_score(grid_search, x_treino, y_treino, cv=outer_cv)

print(f'resultado de cada iteração do cv externo: {dt_nested_scores}')
print(f'média: {dt_nested_scores.mean()}')

grid_search.fit(x_treino, y_treino)

resultado de cada iteração do cv externo: [0.86507937 0.8        0.8       ]
média: 0.8216931216931217


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=3141592, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=3141592), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 6,  8, 10, 12]),
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': array([ 5, 10, 15, 20]),
                         'min_samples_split': array([ 5, 10, 15, 20]),
                         'splitter': ['best', 'random']},
             scoring='recall')

In [17]:
cv_results = pd.DataFrame(grid_search.cv_results_)
display(cv_results.head().T)
###################################################
# melhores hiper-parametros
#display(grid_search.best_params_)

#  melhor modelo
display(cv_results.loc[[grid_search.best_index_]].T)

,0,1,2,3,4
mean_fit_time,0.001294,0.000902,0.001703,0.001063,0.001486
std_fit_time,0.000498,0.000491,0.000399,0.000623,0.000502
mean_score_time,0.001159,0.001604,0.001202,0.001086,0.001253
std_score_time,0.000627,0.000585,0.000399,0.000104,0.000226
param_criterion,gini,gini,gini,gini,gini
param_max_depth,6,6,6,6,6
param_max_features,sqrt,sqrt,sqrt,sqrt,sqrt
param_min_samples_leaf,5,5,5,5,5
param_min_samples_split,5,5,10,10,15
param_splitter,best,random,best,random,best


,71
mean_fit_time,0.003607
std_fit_time,0.003273
mean_score_time,0.002615
std_score_time,0.001033
param_criterion,gini
param_max_depth,8
param_max_features,sqrt
param_min_samples_leaf,5
param_min_samples_split,20
param_splitter,random


# Randomized Search CV (Busca Aleatória) - Otimização de Hiperparâmetros

Randomized Search CV, ou Busca Aleatória, é uma técnica eficaz para otimização de hiperparâmetros em modelos de aprendizado de máquina. Diferente do Grid Search, que testa todas as combinações possíveis de hiperparâmetros, o Randomized Search CV seleciona aleatoriamente um subconjunto dessas combinações, reduzindo o tempo de computação necessário para encontrar uma configuração satisfatória.

**Funcionamento**

1. **Definição do Espaço de Hiperparâmetros:**
   - Um espaço de hiperparâmetros é definido da mesma forma que no Grid Search, mas ao invés de testar todas as combinações possíveis, o Randomized Search CV permite especificar um número fixo de combinações para testar.

2. **Seleção Aleatória:**
   - Combinando um algoritmo de seleção aleatória com uma distribuição de probabilidade definida sobre o espaço de hiperparâmetros, essa técnica seleciona aleatoriamente diferentes combinações para avaliação.

3. **Avaliação de Desempenho:**
   - Cada conjunto de hiperparâmetros selecionado é usado para treinar um modelo, e o desempenho é avaliado, geralmente através de validação cruzada. O processo é repetido para o número especificado de iterações.

4. **Escolha do Melhor Modelo:**
   - Após todas as iterações, o conjunto de hiperparâmetros que resultou no melhor desempenho é escolhido como o ideal.

- **Eficiência:** Menor custo computacional em comparação ao Grid Search, especialmente útil quando o espaço de hiperparâmetros é grande.
- **Exploração mais Ampla:** Possibilidade de descobrir combinações de hiperparâmetros inovadoras que poderiam ser negligenciadas em uma busca exaustiva.
- **Aleatoriedade:** A natureza aleatória da busca significa que não há garantia de encontrar o ótimo global, e os resultados podem variar a cada execução.
- **Número de Iterações:** O número de iterações deve ser escolhido cuidadosamente para equilibrar entre a cobertura do espaço de hiperparâmetros e a viabilidade computacional.


Randomized Search CV é ideal para cenários onde a rapidez é essencial e o espaço de hiperparâmetros é tão grande que uma busca exaustiva seria impraticável. É frequentemente usado em estágios iniciais de desenvolvimento de modelo para explorar rapidamente as possibilidades antes de realizar uma otimização mais detalhada.

Este método é uma ferramenta valiosa para cientistas de dados e engenheiros de machine learning que buscam otimizar modelos de forma eficiente e inovadora.


## in Decision Tree

In [18]:
from sklearn.model_selection import RandomizedSearchCV

grid_search = RandomizedSearchCV(
    estimator= DecisionTreeClassifier(random_state=RANDOM_STATE), #modelo
    param_distributions=param_grid_dt, #grade de parametros
    n_iter=100, #número de vezes que vamos buscar aleatoriamente dentro da grade de hiperparâmetros essas combinações para avaliar o modelo
    scoring='recall',
    n_jobs=-1,
    cv=inner_cv,
    random_state=RANDOM_STATE)

# Validação cruzada externa
rs_dt_nested_score = cross_val_score(grid_search, x_treino, y_treino, cv=outer_cv)

print(f'resultado de cada iteração do cv externo: {rs_dt_nested_score}')
print(f'média: {rs_dt_nested_score.mean()}')

grid_search.fit(x_treino, y_treino)

resultado de cada iteração do cv externo: [0.80952381 0.792      0.8       ]
média: 0.8005079365079366


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=3141592, shuffle=True),
                   estimator=DecisionTreeClassifier(random_state=3141592),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([ 6,  8, 10, 12]),
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': array([ 5, 10, 15, 20]),
                                        'min_samples_split': array([ 5, 10, 15, 20]),
                                        'splitter': ['best', 'random']},
                   random_state=3141592, scoring='recall')

In [19]:
cv_results = pd.DataFrame(grid_search.cv_results_)
display(cv_results.head().T)
###################################################
# melhores hiper-parametros
#display(grid_search.best_params_)

#  melhor modelo
display(cv_results.loc[[grid_search.best_index_]].T)

,0,1,2,3,4
mean_fit_time,0.001,0.001145,0.001906,0.000798,0.001562
std_fit_time,0.000633,0.00019,0.000589,0.000399,0.000935
mean_score_time,0.0012,0.001213,0.001305,0.001398,0.001954
std_score_time,0.0004,0.000394,0.000251,0.000592,0.000519
param_splitter,random,random,best,random,random
param_min_samples_split,15,20,15,15,5
param_min_samples_leaf,5,15,20,10,15
param_max_features,sqrt,sqrt,sqrt,log2,sqrt
param_max_depth,6,6,8,8,6
param_criterion,gini,entropy,entropy,entropy,gini


,3
mean_fit_time,0.000798
std_fit_time,0.000399
mean_score_time,0.001398
std_score_time,0.000592
param_splitter,random
param_min_samples_split,15
param_min_samples_leaf,10
param_max_features,log2
param_max_depth,8
param_criterion,entropy


## in Logistic Regression

In [20]:
grid_search = RandomizedSearchCV(estimator=make_pipeline(StandardScaler(), LogisticRegression()),
        param_distributions=param_grid_lr,
        n_iter=50,
        scoring='recall',
        n_jobs=-1,
        cv=inner_cv,
        random_state=RANDOM_STATE)
        
# Validação cruzada externa
rs_lr_nested_scores = cross_val_score(grid_search, x_treino, y_treino, cv=outer_cv)

print(f'resultado de cada iteração do cv externo: {rs_lr_nested_scores}')
print(f'média: {rs_lr_nested_scores.mean()}')

# Treinando modelo
grid_search.fit(x_treino, y_treino)

resultado de cada iteração do cv externo: [0.93650794 0.968      0.904     ]
média: 0.9361693121693122


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=3141592, shuffle=True),
                   estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('logisticregression',
                                              LogisticRegression())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions=[{'logisticregression__C': [0.001, 0.01,
                                                                   0.1, 1, 10],
                                         'logisticregression__max_iter': array([100, 150, 200, 250, 300]),
                                         'logisticregression__penalty': ['l2'],
                                         'logisticregression__solver': ['newton-cg',
                                                                        'lbfgs']},
                                        {'logisticregression__C': [0.001, 0.01,
                                                                   0.1, 1, 10],
                                         'logisticregression__max_iter': array([100, 150, 200, 250, 300]),
                                         'logisticregression__penalty': ['l1',
                                                                         'l2'],
                                         'logisticregression__solver': ['liblinear']}],
                   random_state=3141592, scoring='recall')

In [21]:
cv_results = pd.DataFrame(grid_search.cv_results_)
display(cv_results.head().T)
###################################################
# melhores hiper-parametros
display(grid_search.best_params_)

#  melhor modelo
display(cv_results.loc[[grid_search.best_index_]].T)

,0,1,2,3,4
mean_fit_time,0.008846,0.00473,0.00728,0.0028,0.005792
std_fit_time,0.001491,0.000735,0.000641,0.000686,0.001606
mean_score_time,0.002017,0.002021,0.002309,0.001921,0.002734
std_score_time,0.000633,0.000362,0.000399,0.001021,0.00086
param_logisticregression__solver,newton-cg,liblinear,newton-cg,liblinear,lbfgs
param_logisticregression__penalty,l2,l1,l2,l1,l2
param_logisticregression__max_iter,300,100,200,200,250
param_logisticregression__C,10,0.1,0.1,0.001,0.1
params,"{'logisticregression__solver': 'newton-cg', 'l...","{'logisticregression__solver': 'liblinear', 'l...","{'logisticregression__solver': 'newton-cg', 'l...","{'logisticregression__solver': 'liblinear', 'l...","{'logisticregression__solver': 'lbfgs', 'logis..."
split0_test_score,0.947368,0.934211,0.934211,0.0,0.934211


{'logisticregression__solver': 'lbfgs',
 'logisticregression__penalty': 'l2',
 'logisticregression__max_iter': 250,
 'logisticregression__C': 0.001}

,14
mean_fit_time,0.005869
std_fit_time,0.001167
mean_score_time,0.002402
std_score_time,0.000492
param_logisticregression__solver,lbfgs
param_logisticregression__penalty,l2
param_logisticregression__max_iter,250
param_logisticregression__C,0.001
params,"{'logisticregression__solver': 'lbfgs', 'logis..."
split0_test_score,0.921053


# Otimização Bayesiana - Otimização de Hiperparâmetros

Diferente dos outros métodos exaustivos, a Otimização Bayesiana utiliza um modelo probabilístico para guiar a busca dos melhores hiperparâmetros, visando maximizar a eficiência ao focar nas áreas mais promissoras do espaço de hiperparâmetros.

- **Eficiência na Busca:** Pode encontrar melhores hiperparâmetros em menos iterações do que métodos exaustivos ou aleatórios.
- **Adaptabilidade:** Ajusta dinamicamente a estratégia de busca com base no que foi aprendido sobre o espaço de hiperparâmetros. Porém...
- **Dependência do Modelo de Prior:** A performance da otimização bayesiana pode depender fortemente da escolha do modelo de prior, que precisa ser bem adequado ao problema.

**Funcionamento**

1. **Modelo Probabilístico:**
   - A técnica começa com a construção de um modelo probabilístico do espaço de hiperparâmetros. Este modelo, frequentemente um Processo Gaussiano, estima a função de desempenho do modelo de aprendizado de máquina como uma distribuição de probabilidade sobre o espaço de hiperparâmetros.

2. **Atualizações Sequenciais:**
   - Com cada nova avaliação de hiperparâmetros, o modelo probabilístico é atualizado. A técnica utiliza o conceito de "aquisição" para decidir quais hiperparâmetros devem ser avaliados a seguir, baseando-se na probabilidade de melhoria.

3. **Função de Aquisição:**
   - A função de aquisição, como Expected Improvement (EI), Upper Confidence Bound (UCB), ou Probability of Improvement (PI), ajuda a determinar o próximo conjunto de hiperparâmetros a ser testado. Essa função equilibra entre explorar novas áreas do espaço e explorar em torno de hiperparâmetros que já mostraram bons resultados.

4. **Seleção do Melhor Modelo:**
   - O processo continua até que um critério de parada específico seja alcançado, seja ele um número fixo de iterações, um tempo limite, ou uma convergência no desempenho do modelo.

In [22]:
#pip install scikit-optimize

## in Decision Tree

In [23]:
# Grade de hiperparametro 
# é definido uma forma diferente....
from skopt.space import Real, Integer, Categorical
space_dt = {
    'criterion': Categorical(['gini', 'entropy']),
    'max_depth': Integer(6, 12),
    'min_samples_split': Integer(5, 20),
    'min_samples_leaf': Integer(5, 20),
    'max_features': Categorical(['sqrt', 'log2']),
    'splitter': Categorical(['best', 'random'])
}

In [24]:
from skopt import BayesSearchCV

opt_dt = BayesSearchCV(
    estimator=DecisionTreeClassifier(random_state=RANDOM_STATE),
    search_spaces = space_dt, 
    n_iter=50, # numero de interações
    scoring='recall', 
    n_jobs=-1, 
    cv=inner_cv, 
    random_state=RANDOM_STATE
    )

# Validação cruzada aninhada
bs_dt_nested_scores = cross_val_score(opt_dt, x_treino, y_treino, cv=outer_cv)

print(f'resultado de cada iteração do cv externo: {bs_dt_nested_scores}')
print(f'média: {bs_dt_nested_scores.mean()}')

opt_dt.fit(x_treino, y_treino)

resultado de cada iteração do cv externo: [0.84126984 0.856      0.792     ]
média: 0.8297566137566138


BayesSearchCV(cv=StratifiedKFold(n_splits=5, random_state=3141592, shuffle=True),
              estimator=DecisionTreeClassifier(random_state=3141592), n_jobs=-1,
              random_state=3141592, scoring='recall',
              search_spaces={'criterion': Categorical(categories=('gini', 'entropy'), prior=None),
                             'max_depth': Integer(low=6, high=12, prior='uniform', transform='normalize'),
                             'max_features': Categorical(categories=('sqrt', 'log2'), prior=None),
                             'min_samples_leaf': Integer(low=5, high=20, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=5, high=20, prior='uniform', transform='normalize'),
                             'splitter': Categorical(categories=('best', 'random'), prior=None)})

In [25]:
print(f'resultado de cada iteração do cv externo: {bs_dt_nested_scores}')
print(f'média: {bs_dt_nested_scores.mean()}')

resultado de cada iteração do cv externo: [0.84126984 0.856      0.792     ]
média: 0.8297566137566138


In [26]:
cv_results = pd.DataFrame(opt_dt.cv_results_)
display(cv_results.head().T)
###################################################
# melhores hiper-parametros
#display(opt_dt.best_params_)

#  melhor modelo
display(cv_results.loc[[opt_dt.best_index_]].T)

,0,1,2,3,4
mean_fit_time,0.002661,0.001389,0.002129,0.002157,0.004338
std_fit_time,0.000419,0.000479,0.000665,0.000312,0.003582
mean_score_time,0.001202,0.001257,0.001405,0.001103,0.001061
std_score_time,0.000399,0.000323,0.000503,0.000489,0.000261
param_criterion,entropy,entropy,gini,entropy,gini
param_max_depth,7,10,6,9,6
param_max_features,log2,log2,log2,sqrt,log2
param_min_samples_leaf,13,6,12,9,18
param_min_samples_split,13,8,14,18,14
param_splitter,best,random,best,best,best


,17
mean_fit_time,0.001333
std_fit_time,0.000387
mean_score_time,0.001147
std_score_time,0.000145
param_criterion,gini
param_max_depth,12
param_max_features,sqrt
param_min_samples_leaf,5
param_min_samples_split,13
param_splitter,random


## in Logistic Regression

In [27]:
# Definindo Grade
max_iter = Integer(100, 300)
c = Categorical([0.001, 0.01, 0.1, 1, 10])

space_lr = [
    {
        'logisticregression__solver': Categorical(['newton-cg', 'lbfgs']),
        'logisticregression__penalty': Categorical(['l2']),
        'logisticregression__max_iter': max_iter,
        'logisticregression__C': c
    },
    {
        'logisticregression__solver': Categorical(['liblinear']),
        'logisticregression__penalty': Categorical(['l1', 'l2']),
        'logisticregression__max_iter': max_iter,
        'logisticregression__C': c
    },
]

In [28]:
opt_lr = BayesSearchCV(
    estimator=make_pipeline(StandardScaler(), LogisticRegression()),
    search_spaces = space_lr,
    n_iter=50,
    scoring='recall',
    n_jobs=-1,
    cv=inner_cv,
    random_state=RANDOM_STATE)
            
# Validação cruzada aninhada
bs_lr_nested_scores = cross_val_score(opt_lr, x_treino, y_treino, cv=outer_cv)

print(f'resultado de cada iteração do cv externo: {bs_lr_nested_scores}')
print(f'média: {bs_lr_nested_scores.mean()}')

opt_dt.fit(x_treino, y_treino)

resultado de cada iteração do cv externo: [0.93650794 0.968      0.904     ]
média: 0.9361693121693122


BayesSearchCV(cv=StratifiedKFold(n_splits=5, random_state=3141592, shuffle=True),
              estimator=DecisionTreeClassifier(random_state=3141592), n_jobs=-1,
              random_state=3141592, scoring='recall',
              search_spaces={'criterion': Categorical(categories=('gini', 'entropy'), prior=None),
                             'max_depth': Integer(low=6, high=12, prior='uniform', transform='normalize'),
                             'max_features': Categorical(categories=('sqrt', 'log2'), prior=None),
                             'min_samples_leaf': Integer(low=5, high=20, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=5, high=20, prior='uniform', transform='normalize'),
                             'splitter': Categorical(categories=('best', 'random'), prior=None)})

In [29]:
cv_results = pd.DataFrame(opt_dt.cv_results_)
display(cv_results.head().T)
###################################################
# melhores hiper-parametros
#display(opt_dt.best_params_)

#  melhor modelo
display(cv_results.loc[[opt_dt.best_index_]].T)

,0,1,2,3,4
mean_fit_time,0.002164,0.001207,0.001807,0.0022,0.001419
std_fit_time,0.000242,0.000176,0.000421,0.000401,0.000511
mean_score_time,0.000981,0.001632,0.001242,0.001603,0.001615
std_score_time,0.000284,0.00038,0.0008,0.000487,0.000508
param_criterion,entropy,entropy,gini,entropy,gini
param_max_depth,7,10,6,9,6
param_max_features,log2,log2,log2,sqrt,log2
param_min_samples_leaf,13,6,12,9,18
param_min_samples_split,13,8,14,18,14
param_splitter,best,random,best,best,best


,17
mean_fit_time,0.001471
std_fit_time,0.000448
mean_score_time,0.001401
std_score_time,0.000487
param_criterion,gini
param_max_depth,12
param_max_features,sqrt
param_min_samples_leaf,5
param_min_samples_split,13
param_splitter,random


Entendendo qual técnica usar
Após apresentar essas técnicas de otimização bayesiana, surge a pergunta: qual delas escolher? A resposta é: depende. A escolha depende do seu propósito e da sua situação.

Se você possui um `conjunto de dados não tão grande ou um espaço de busca limitado, o Grid Search pode ser uma opção.`
Para conjuntos de dados extensos, a busca aleatória pode ser uma alternativa.

No entanto, se o espaço de busca é vasto, como de 100 a 300, e utilizar o Randomized Search ou o Grid Search implicaria em uma variação grande, por exemplo, de 10 em 10, pode ser mais eficaz considerar a otimização bayesiana.

Assim, teria mais ou menos uns 200 opções só para o max_depth. Utilizando a otimização bayesiana, ele vai escolher valores entre 100 e 300, por exemplo. Portanto, se tiver espaços de busca muito elevados, com muitos valores, talvez a otimização bayesiana seja uma melhor opção, já que ela também aprende com os resultados anteriores.

# FIM